# Create a virtual display 🔽

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Install dependencies 🔽

In [3]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.4 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=248f6d9005926d75c8a25dc55b1dd3d966767cd18f4e6258209cf86d392beb2f
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.10 require

# Import the packages 📦

In [4]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import login

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


# PandaReachDense-v3 🦾
The agent we’re going to train is a robotic arm that needs to do controls (moving the arm and using the end-effector).

In robotics, the end-effector is the device at the end of a robotic arm designed to interact with the environment.

In PandaReach, the robot must place its end-effector at a target position (green ball).

We’re going to use the dense version of this environment. It means we’ll get a dense reward function that will provide a reward at each timestep (the closer the agent is to completing the task, the higher the reward). Contrary to a sparse reward function where the environment return a reward if and only if the task is completed.

Also, we’re going to use the End-effector displacement control, it means the action corresponds to the displacement of the end-effector. We don’t control the individual motion of each joint (joint control).

## The environment 🎮

In [5]:
env_id = "PandaReachDense-v3"

env = gym.make(env_id)

s_size = env.observation_space.shape
a_size = env.action_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([8.114315 , 1.5747985, 5.0605803], dtype=float32)), ('desired_goal', array([-8.458551 , -9.493125 ,  3.1096568], dtype=float32)), ('observation', array([-6.698305 , -4.4603224, -4.3671637, -9.454221 ,  6.4856853,
       -7.460611 ], dtype=float32))])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The observation space is a dictionary with 3 different elements:

- achieved_goal: (x,y,z) position of the goal.
- desired_goal: (x,y,z) distance between the goal position and the current object position.
- observation: position (x,y,z) and velocity of the end-effector (vx, vy, vz).

In [7]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.6445129   0.71101964 -0.51594174]


The action space is a vector with 3 values:

- Control x, y, z movement

## Normalize observation and rewards

In [8]:
env = make_vec_env(env_id, n_envs=4)

# Adding this wrapper to normalize the observation and the reward
env = VecNormalize(env, norm_obs=True, norm_reward=True)

# Create the A2C Model 🤖

In [9]:
model = A2C("MultiInputPolicy", env, device="cpu", verbose=1)

Using cpu device


In [10]:
model.learn(1_000_000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 44.1     |
|    ep_rew_mean        | -12.7    |
| time/                 |          |
|    fps                | 357      |
|    iterations         | 100      |
|    time_elapsed       | 5        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.3     |
|    explained_variance | 0.814    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.918    |
|    std                | 1.01     |
|    value_loss         | 0.278    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 43       |
|    ep_rew_mean        | -11.9    |
| time/                 |          |
|    fps                | 360      |
|    iterations         | 200      |
|    time_elapsed       | 11       |
|    total_timesteps    | 4000     |
|

In [11]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

# Evaluate the agent 📈

In [12]:
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"
# do not update at test time
eval_env.training = False
# no need to normalize reward at test time
eval_env.norm_reward = False

model = A2C.load("a2c-PandaReachDense-v3")
mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward: -0.25 +/- 0.12


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_content, map_locati

# Publish your trained model on the Hub 🔥

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"wowthecoder/a2c-{env_id}", 
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpd9ly021e/-step-0-to-step-1000.mp4


/usr/local/lib/python3.10/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """


Moviepy - Building video /tmp/tmpd9ly021e/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpd9ly021e/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpd9ly021e/-step-0-to-step-1000.mp4
ℹ Pushing repo wowthecoder/a2c-PandaReachDense-v3 to the Hugging Face
Hub


policy.optimizer.pth:   0%|          | 0.00/48.2k [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/46.3k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/wowthecoder/a2c-PandaReachDense-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/wowthecoder/a2c-PandaReachDense-v3/commit/02bec1dc136955edadacbdff33f6faba2944fc39', commit_message='Initial commit', commit_description='', oid='02bec1dc136955edadacbdff33f6faba2944fc39', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wowthecoder/a2c-PandaReachDense-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='wowthecoder/a2c-PandaReachDense-v3'), pr_revision=None, pr_num=None)